In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, datasets, transforms
from torchsummary import summary
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset 
import numpy as np
import torch.optim as optim

In [ ]:
VERSION = "20200325"  #@param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4264  100  4264    0     0  46857      0 --:--:-- --:--:-- --:--:-- 46857
Updating TPU and VM. This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200325 ...
Uninstalling torch-1.5.1+cu101:
Done updating TPU runtime: <Response [200]>
  Successfully uninstalled torch-1.5.1+cu101
Uninstalling torchvision-0.6.1+cu101:
  Successfully uninstalled torchvision-0.6.1+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly+20200325-cp36-cp36m-linux_x86_64.whl...
- [1 files][ 83.4 MiB/ 83.4 MiB]                                                
Operation completed over 1 objects/83.4 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-nightly+20200325-cp36-cp36m-linux_x86_64.whl...
- [1 files][114.5 MiB/114.5 MiB]                                                
Operation completed over 1 objec

In [ ]:
import torch_xla
import torch_xla.core.xla_model as xm
device = xm.xla_device()

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"huyngoc99168","key":"0a0c29dfa8d1be7f16234feeb7b9e830"}'}

In [ ]:
!cp /content/kaggle.json "/content/kaggle"

In [ ]:
!chmod 600 /content/kaggle/kaggle.json

In [ ]:
resnet50 = models.resnet50(pretrained = True)
for para in resnet50.parameters():
  para.requires_grad = False

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


In [ ]:
num_feature = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_feature, 42)
for para in resnet50.fc.parameters():
  para.requires_grad = True
for para in resnet50.layer4.parameters():
  para.requires_grad = True

In [ ]:
resnet50.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
transform = transforms.Compose([transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])])
Ds_train = datasets.ImageFolder("/content/Cac/train/train", transform = transform)
Dl_train = DataLoader(Ds_train, batch_size = 64, shuffle = True)

In [ ]:
loss = nn.CrossEntropyLoss()
parameters = filter(lambda x: x.requires_grad, resnet50.parameters())
optimizer = optim.SGD(parameters, lr = 0.001)

In [ ]:
!pip install barbar

In [ ]:
from barbar import Bar
resnet50.load_state_dict(torch.load("/content/drive/My Drive/weight_19.pth"))

<All keys matched successfully>

In [ ]:
def count(model):
  return sum([p.numel() for p in resnet50.parameters() if p.requires_grad])
count(resnet50)

15050794

In [ ]:
Loss = []
Acc = []
Epoch = []
lr = 0.002
num_epoch = 15
parameters = filter(lambda x: x.requires_grad , resnet50.parameters())
optimizer = optim.SGD(parameters, lr = lr)
for epoch in range(num_epoch):
    sum_loss = 0
    total = 0
    correct = 0
    print('Epoch: %d/%d' % (epoch + 1, num_epoch))
    for X, y in Bar(Dl_train):
        y.long()
        X = X.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        y_pred = resnet50(X)
        loss = F.cross_entropy(y_pred, y)
        loss.backward()
        xm.optimizer_step(optimizer, barrier=True)
        total += y.size(0)
        sum_loss += loss.item()*(y.size(0))
        _, predicted = torch.max(y_pred, 1)
        correct += (predicted == y).sum().item()
    Loss.append(sum_loss/total)
    Acc.append(correct/total)
    Epoch.append(epoch)
    print("'Loss: {} ----- Accuracy: {} %".format(sum_loss/total, (correct*100)/total))
    if (epoch % 2) == 0:
      ggdr_path = "/content/drive/My Drive/weight_" + str(epoch+20) + ".pth"
      torch.save(resnet50.state_dict(), ggdr_path)
print("The training process is done")
torch.save(resnet50, model_path)
print("model saved")
torch.save(resnet50.state_dict(), weight_path)

Epoch: 1/15
105392/105392: [===============================>] - ETA 8.7s
'Loss: 1.0429338226554352 ----- Accuracy: 71.1420221648702 %
Epoch: 2/15
105392/105392: [===============================>] - ETA 8.6s
'Loss: 0.9911155266520719 ----- Accuracy: 72.31478670107788 %
Epoch: 3/15
105392/105392: [===============================>] - ETA 1.1s
'Loss: 0.9387821988305137 ----- Accuracy: 73.77220282374374 %
Epoch: 4/15
105392/105392: [===============================>] - ETA 1.2s
'Loss: 0.8854583964120533 ----- Accuracy: 75.37004706239563 %
Epoch: 5/15
105392/105392: [===============================>] - ETA 1.2s
'Loss: 0.8353998499791105 ----- Accuracy: 76.65192804007894 %
Epoch: 6/15
105392/105392: [===============================>] - ETA 1.2s
'Loss: 0.7802436374760152 ----- Accuracy: 78.32947472293912 %
Epoch: 7/15
105392/105392: [===============================>] - ETA 1.2s
'Loss: 0.7241461234297578 ----- Accuracy: 80.04117959617429 %
Epoch: 8/15
105392/105392: [============================

NameError: ignored